# homuncu_loc image viewer

In [1]:
import napari
from skimage import io
import btrack

In [ ]:
image_ID = '20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1365'

In [18]:
image_fn = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/Job_Mtb area/run2_23-02-104/48h pi/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1365.tif'

In [19]:
%%time
image = io.imread(image_fn)

CPU times: user 1.17 s, sys: 4.29 s, total: 5.46 s
Wall time: 1min 2s


In [9]:
image.shape

(93, 2304, 2304, 4)

In [16]:
tracks_fn = '/home/dayn/data/homuncu_loc_temp/run2_23-02-104/48h pi/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1365_z_tracks_masks.h5'

In [17]:
with btrack.io.HDF5FileHandler(tracks_fn, 'r', obj_type='obj_type_1') as reader:
    segmentation = reader.segmentation
    tracks = reader.tracks

[INFO][2023/08/04 03:58:10 pm] Opening HDF file: /home/dayn/data/homuncu_loc_temp/run2_23-02-104/48h pi/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1365_z_tracks_masks.h5...
04-Aug-23 15:58:10 - btrack.io.hdf - INFO     - Opening HDF file: /home/dayn/data/homuncu_loc_temp/run2_23-02-104/48h pi/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1365_z_tracks_masks.h5...
[INFO][2023/08/04 03:58:12 pm] Loading segmentation (81, 2304, 2304)
04-Aug-23 15:58:12 - btrack.io.hdf - INFO     - Loading segmentation (81, 2304, 2304)
[INFO][2023/08/04 03:58:12 pm] Loading tracks/obj_type_1
04-Aug-23 15:58:12 - btrack.io.hdf - INFO     - Loading tracks/obj_type_1
[INFO][2023/08/04 03:58:12 pm] Loading LBEP/obj_type_1
04-Aug-23 15:58:12 - btrack.io.hdf - INFO     - Loading LBEP/obj_type_1
[INFO][2023/08/04 03:58:12 pm] Loading objects/obj_type_1 (34822, 5) (34822 filtered: None)
04-Aug-23 15:58:12 - btrack.io.hdf - INFO     - Loading objects/obj_type_1 (34822, 5) (34822 filtered: None)

In [20]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)

In [21]:
recolored_segmentation = btrack.utils.update_segmentation(segmentation, tracks, color_by='ID')

In [22]:
viewer = napari.Viewer()

viewer.add_image(image, channel_axis = -1)
viewer.add_labels(segmentation, visible = False)
viewer.add_labels(recolored_segmentation)
viewer.add_tracks(napari_tracks)

<Tracks layer 'napari_tracks' at 0x7f8c41824760>

In [23]:
print()